---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
from matplotlib import pyplot as plt

def graph_identification():
    
#      for j in range(len(P1_Graphs)):
#         G = P1_Graphs[j]
#         all_degrees = G.degree()
#         main_degree = sorted(set(degrees.values()))
#         histogram = [sum(value == v for value in all_degrees.values()) / float(nx.number_of_nodes(G)) for v in main_degree]
        
        
#         plt.plot(degree_values, histogram,'o')
#         plt.xlabel('Degree')
#         plt.ylabel('Fraction of nodes')
#         plt.xscale('log')
#         plt.yscale('log')
#         plt.show()




# for graph in P1_Graphs:
#     print('Average_Clustering=',nx.average_clustering(graph))
    
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']
    




---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [8]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))


Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [4]:
def salary_predictions():
    
    nodes_features=G.nodes(data=True)

    df=pd.DataFrame(nodes_features, columns=['Node', 'Features'])

    df['Department']=df['Features'].map(lambda x: x['Department'])
    df['ManagementSalary']=df['Features'].map(lambda x: x['ManagementSalary'])
    df.drop('Features',axis=1,inplace=True)
    df.set_index('Node',inplace=True)

    df['closeness_centrality']=pd.Series(nx.closeness_centrality(G))
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['betweenness_centrality'] = pd.Series(nx.betweenness_centrality(G))
    df['Clustering'] = pd.Series(nx.clustering(G))
    df['PageRank'] = pd.Series(nx.pagerank(G))
    df['Hub'] = pd.Series(nx.hits(G)[0])
    df['Authority'] = pd.Series(nx.hits(G)[1])

    testdf=df[df['ManagementSalary'].isnull()]
    test_X_All=testdf.drop(['ManagementSalary'],axis=1)
    traindf_All=df[df['ManagementSalary'].isnull()==False]

    train_X_All=traindf_All.drop(['ManagementSalary'],axis=1)
    train_y_All=traindf_All['ManagementSalary']


    model=RandomForestClassifier()
    # np.mean(cross_val_score(model, train_X_All, train_y_All, cv=10,scoring='roc_auc'))

    model.fit(train_X_All,train_y_All)
    pred=model.predict(test_X_All)
    pred=pd.Series(model.predict_proba(test_X_All)[:,1],index=test_X_All.index)
    
    
    return pred


### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [5]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})



Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [10]:
def new_connections_predictions():
    
    idx=future_connections.index
    future_connections['comm_neigh']=[len(list(nx.common_neighbors(G,i[0],i[1]))) for i in idx]

    future_connections['pref_att']=[i[2] for i in nx.preferential_attachment(G,idx)]
    future_connections['jacc_Coed']=[i[2] for i in nx.jaccard_coefficient(G,idx)]
    future_connections['res_all']=[i[2] for i in nx.resource_allocation_index(G,idx)]
    future_connections['ad_ad']=[i[2] for i in nx.adamic_adar_index(G,idx)]

    future_connections['sound_hop']=[i[2] for i in nx.cn_soundarajan_hopcroft(G,idx,'Department')]
    future_connections['ra_sound_hop']=[i[2] for i in nx.ra_index_soundarajan_hopcroft(G,idx,'Department')]

    future_connections['Future Connection']
    testdf=future_connections[future_connections['Future Connection'].isnull()]
    test_X_All=testdf.drop(['Future Connection'],axis=1)

    traindf_All=future_connections[future_connections['Future Connection'].isnull()==False]

    train_X_All=traindf_All.drop(['Future Connection'],axis=1)
    train_y_All=traindf_All['Future Connection']


    model=RandomForestClassifier(n_estimators=300,max_depth=12)
#     grid_param={'n_estimators':[100,200,300,400],'max_depth':[5,6,7,8,9,10,11,12]}
#     np.mean(cross_val_score(model, train_X_All, train_y_All, cv=3,scoring='roc_auc'))
#     searchclf=RandomizedSearchCV(model,grid_param,n_iter=10,cv=3,scoring='roc_auc',verbose=2)
#     searchclf.fit(train_X_All, train_y_All)

#     print(searchclf.best_params_)
#     print(searchclf.best_score_)
    model.fit(train_X_All,train_y_All)
    pred=model.predict(test_X_All)
    pred=pd.Series(model.predict_proba(test_X_All)[:,1],index=test_X_All.index)
    
    return pred




   